# Чтение Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

## Чтение с локального диска `CSV`

In [ ]:
df = spark.read.csv("/shared_data/bmw.csv", 
                    header=True, 
                    inferSchema=True)

In [ ]:
# Чтение всех файлов из папки
# ВАЖНО! Можно читать только одинаковые по структуре файлы

df = spark.read.csv("/shared_data/*.csv", 
                    header=True)

In [ ]:
# Чтение с параметрами
df = (spark.read
      .option("header", "true")
      .option("delimiter", ",")
      .option("inferSchema", "true")
      .option("encoding", "utf-8")
      .option("nullValue", "NULL")
      .csv("/shared_data/bmw.csv"))

In [ ]:
# Использование формата
df = (spark.read
      .format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load("/shared_data/bmw.csv"))

## Чтение с указанием схемы

In [ ]:
# Чтение CSV с указанием схемы
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Определяем схему для CSV файла
schema = StructType([
    StructField("Model",                StringType(),  True),
    StructField("Year",                 IntegerType(), True),
    StructField("Region",               StringType(),  True),
    StructField("Color",                StringType(),  True),
    StructField("Fuel_Type",            StringType(),  True),
    StructField("Transmission",         StringType(),  True),
    StructField("Engine_Size_L",        DoubleType(),  True),
    StructField("Mileage_KM",           IntegerType(), True),
    StructField("Price_USD",            IntegerType(), True),
    StructField("Sales_Volume",         IntegerType(), True),
    StructField("Sales_Classification", StringType(),  True)
])
            
df = spark.read.csv("/shared_data/bmw.csv",
                    schema=schema,
                    header=True)

## Запись в MinIO `CSV`

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import json

(df.write
   .mode("overwrite")
   .option("header", "false")
   .csv("s3a://learn-bucket/draft/bmw_csv"))

# Спарк по умолчанию не пишет заголовки в CSV файлы
# По этому можно сохранить схему прямо в MinIO

schema_json = df.schema.json()
schema_df = spark.createDataFrame([(schema_json,)], ["schema"])

schema_df.write \
    .mode("overwrite") \
    .option("compression", "none") \
    .text("s3a://learn-bucket/draft/bmw_schema")

## Чтение из MinIO `CSV`

In [ ]:
# Читаем схему напрямую из MinIO
schema_rdd = spark.sparkContext.textFile("s3a://learn-bucket/draft/bmw_schema")
schema_json = schema_rdd.collect()[0]  # берем первую строку

# Восстанавливаем схему
restored_schema = StructType.fromJson(json.loads(schema_json))

# Теперь читаем данные с этой схемой
df = (spark.read
           .schema(restored_schema)
           .option("header", "false")
           .csv("s3a://learn-bucket/draft/bmw_csv"))

# Parquet

### С локального диска

In [ ]:
df = spark.read.parquet("/shared_data/yellow_taxi")

In [ ]:
df = spark.read.parquet("/shared_data/yellow_taxi/data_*.parquet")

In [ ]:
df = spark.read.parquet("/shared_data/yellow_taxi/single_file.parquet")

In [ ]:
print(f"Количество строк: {df.count()}\n")
df.show(1)
df.printSchema()

### Из MinIO

In [ ]:
spark.stop()

# Postgres

In [ ]:
jdbc_url = "jdbc:postgresql://postgres-db:5432/learn_base"

postgres_con = {
    "user": "airflow",
    "password": "airflow", 
    "driver": "org.postgresql.Driver"
}
# Пример 1: Читаем данные
df = (spark.read.jdbc(url=jdbc_url, 
                      table="какой_кайф", 
                      properties=postgres_con))

df.show()

# Варианты чтения данных из PostgreSQL в Spark DataFrame

## 1. Базовый вариант
```python
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("dbtable", "table_name") \
    .option("user", "username") \
    .option("password", "password") \
    .load()
```

## 2. Короткий вариант через jdbc()
```python
df = spark.read.jdbc(
    url="jdbc:postgresql://localhost:5432/mydb",
    table="table_name",
    properties={
        "user": "username",
        "password": "password", 
        "driver": "org.postgresql.Driver"
    }
)
```

## 3. Чтение с SQL запросом
```python
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("user", "username") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .option("query", "SELECT * FROM users WHERE age > 18 AND city = 'Moscow'") \
    .load()
```

## 4. Параллельное чтение с партиционированием
```python
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("dbtable", "large_table") \
    .option("user", "username") \
    .option("password", "password") \
    .option("partitionColumn", "id") \     # колонка для партиционирования
    .option("lowerBound", "1") \           # минимальное значение
    .option("upperBound", "1000000") \     # максимальное значение  
    .option("numPartitions", "10") \       # количество партиций
    .load()
```

## 5. Чтение с фильтрацией через predicates
```python
predicates = [
    "date >= '2024-01-01' AND date < '2024-02-01'",  # партиция 1
    "date >= '2024-02-01' AND date < '2024-03-01'",  # партиция 2
    "date >= '2024-03-01' AND date < '2024-04-01'"   # партиция 3
]

df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("dbtable", "sales") \
    .option("user", "username") \
    .option("password", "password") \
    .option("predicates", predicates) \
    .load()
```

## 6. Чтение с кастомными настройками
```python
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("dbtable", "table_name") \
    .option("user", "username") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .option("fetchsize", "10000") \          # размер fetch для больших данных
    .option("sessionInitStatement", "SET TIME ZONE 'UTC'") \  # инициализация сессии
    .option("customSchema", "id DECIMAL(38,0), name STRING") \  # кастомные типы
    .load()
```

## 7. Чтение только определенных колонок
```python
# Через SQL запрос
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("user", "username") \
    .option("password", "password") \
    .option("query", "SELECT id, name, email FROM users WHERE active = true") \
    .load()

# Через dbtable с подзапросом
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/mydb") \
    .option("user", "username") \
    .option("password", "password") \
    .option("dbtable", "(SELECT id, name FROM users) AS users_subset") \
    .load()
```

## 8. Полный пример с обработкой ошибок
```python
def read_from_postgres(table_name_or_query, **options):
    try:
        base_options = {
            "url": "jdbc:postgresql://localhost:5432/mydb",
            "user": "username",
            "password": "password",
            "driver": "org.postgresql.Driver"
        }
        base_options.update(options)
        
        # Определяем, это таблица или запрос
        if "SELECT" in table_name_or_query.upper():
            base_options["query"] = table_name_or_query
        else:
            base_options["dbtable"] = table_name_or_query
            
        df = spark.read.format("jdbc").options(**base_options).load()
        
        print(f"✅ Данные прочитаны: {len(df.columns)} колонок, {df.count()} строк")
        return df
        
    except Exception as e:
        print(f"❌ Ошибка чтения: {str(e)}")
        return None

# Использование
df1 = read_from_postgres("users")
df2 = read_from_postgres("SELECT * FROM orders WHERE status = 'completed'")
df3 = read_from_postgres("large_table", numPartitions=10, partitionColumn="id")
```

## 9. Универсальная функция чтения
```python
def postgres_to_spark(source, **options):
    """
    source: имя таблицы или SQL запрос
    """
    base_config = {
        "url": "jdbc:postgresql://localhost:5432/mydb",
        "user": "username",
        "password": "password",
        "driver": "org.postgresql.Driver",
        "fetchsize": 10000
    }
    base_config.update(options)
    
    reader = spark.read.format("jdbc")
    
    if "SELECT" in source.upper():
        reader.option("query", source)
    else:
        reader.option("dbtable", source)
    
    return reader.options(**base_config).load()

# Примеры использования
df_table = postgres_to_spark("products")
df_query = postgres_to_spark("SELECT * FROM sales WHERE amount > 1000")
df_partitioned = postgres_to_spark(
    "large_table",
    partitionColumn="id",
    lowerBound=1,
    upperBound=1000000,
    numPartitions=8
)
```

Все варианты готовы к использованию! 🚀